In [ ]:
import pandas as pd
from astroquery.gaia import Gaia
import numpy as np
import plotly.express as px

# Gaia sorgusu için RA ve DEC aralıkları
ra_bins = [(0, 60), (60, 120), (120, 180), (180, 240), (240, 300), (300, 360)]
dec_bins = [(-30, -10), (-10, 10), (10, 30)]

all_results = pd.DataFrame()

for ra_min, ra_max in ra_bins:
    for dec_min, dec_max in dec_bins:
        query = f"""
        SELECT source_id, ra, dec, phot_g_mean_mag, bp_rp, parallax
        FROM gaiadr3.gaia_source
        WHERE parallax > 10
        AND phot_g_mean_mag < 15
        AND bp_rp IS NOT NULL
        AND ra BETWEEN {ra_min} AND {ra_max}
        AND dec BETWEEN {dec_min} AND {dec_max}
        """
        print(f"Sorgulanıyor: RA {ra_min}-{ra_max}, DEC {dec_min}-{dec_max}")
        job = Gaia.launch_job(query)
        results = job.get_results().to_pandas()
        all_results = pd.concat([all_results, results], ignore_index=True)

# Mesafe ve mutlak parlaklık hesaplama
all_results['distance_pc'] = 1000 / all_results['parallax']
all_results['abs_mag'] = all_results['phot_g_mean_mag'] - 5 * (np.log10(all_results['distance_pc']) - 1)

print(f"Toplam çekilen yıldız sayısı: {len(all_results)}")

# Interaktif HR diyagramı çizimi
fig = px.scatter(
    all_results,
    x='bp_rp',
    y='abs_mag',
    labels={'bp_rp': 'BP - RP (Color Index)', 'abs_mag': 'Absolute Magnitude (G Band)'},
    title='Interactive Gaia HR Diagram',
    color='abs_mag',
    color_continuous_scale='plasma_r',
    hover_data=['source_id', 'ra', 'dec']
)

fig.update_yaxes(autorange='reversed')
fig.show()

Sorgulanıyor: RA 0-60, DEC -30--10
Sorgulanıyor: RA 0-60, DEC -10-10
Sorgulanıyor: RA 0-60, DEC 10-30
